***
## Новые поля и миграции

Начнём с изменения структуры базы данных. Поле `output_order` уже есть в модели `Category`, и его можно просто скопировать в модель `IceCream`:

```py
class IceCream(PublishedModel):
    ...

    output_order = models.PositiveSmallIntegerField(
        default=100,
        verbose_name='Порядок отображения'
    )

    ... 
```

Для поля `price` хорошей идеей будет воспользоваться типом `DecimalField`.

Этот тип ожидает два обязательных аргумента:

* `DecimalField.max_digits` — максимальное количество цифр, допустимое в числе.

* `DecimalField.decimal_places` — Количество десятичных знаков, которое должно храниться в числе.

Детальное описание этого типа [есть в документации](https://docs.djangoproject.com/en/3.2/ref/models/fields/#decimalfield).

Обратите внимание, что значение `max_digits` должно быть больше или равно `decimal_places`. Например, для хранения чисел до 99 с указанием двух знаков после запятой можно написать так: `models.DecimalField(max_digits=4, decimal_places=2)`.

После внесения изменений в модель необходимо создать и применить миграции. Однако в процессе их создания Django выдаст следующее предупреждение:

```bash
python manage.py makemigrations
You are trying to add a non-nullable field 'price' to icecream without a default; we can't do that (the database needs something to populate existing rows).
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows with a null value for this column)
 2) Quit, and let me add a default in models.py
Select an option: 1
Please enter the default value now, as valid Python
The datetime and django.utils.timezone modules are available, so you can do e.g. timezone.now
Type 'exit' to exit this prompt 
```

В таблицу базы данных добавляются два обязательных поля; в таблице уже есть записи, и для всех существующих записей в новых полях должно быть указано какое-то значение. 

И Django ORM спрашивает: 

```
Что делать? С полем output_order проблем нет, 
для него указано значение (default=100), но что делать с полем price? 
Оно не может быть пустым, а дефолтное значение для него не указано. 
И какое же значение присвоить этому полю?  
```

Django ORM предлагает два варианта выхода из этой ситуации: 

* продолжить создание миграций, указав нужное значение прямо в консоли,

* прервать создание миграций, указать дефолтное значение поля в модели — а потом заново выполнить `makemigrations`.

Первый вариант вполне подходит для решения задачи.

В консоли введите 1 и нажмите Enter. После этого будет запрошено значение, которое нужно добавить в поле `price` при его добавлении к существующим записям; укажите 0 — и проблема решена.

```bash
Select an option: 1
Please enter the default value now, as valid Python
The datetime and django.utils.timezone modules are available, so you can do e.g. timezone.now
Type 'exit' to exit this prompt
>>> 0
Migrations for 'ice_cream':
  ice_cream/migrations/0002_auto_20220904_0933.py
    - Add field price to icecream
    - Add field output_order to icecream 
```

***
## Новые view-функции и шаблоны

На разные страницы сайта выводятся разные данные из базы данных, например:

* название мороженого,

* описание мороженого,

* стоимость мороженого,

* название обёртки мороженого,

* название категории, к которой принадлежит мороженое,

* названия всех топпингов, связанных с мороженым.

В новых шаблонах, из которых рендерится главная страница, страница каталога и страница отдельного мороженого, уже указаны нужные переменные словаря `context`. Чтобы правильно составить ORM-запросы к базе данных, посмотрим внимательно на эти шаблоны.

Начнем работу с главной страницы — с шаблона *index.html*. 

В коде шаблона страницы нужна только одна переменная — QuerySet `ice_cream_list`. К шаблону также подключена карточка мороженого, которая вынесена в отдельный файл *ice_cream_card.html*.

Из HTML-кода видно, что для каждого мороженого из БД понадобится запросить такие поля:

* `id`

* `title`

* `price`

* `description`

Также в запросе необходимо добавить проверку флагов

* `is_published` для сорта мороженого,

* `is_published` для категории,

* `is_on_main` для сорта мороженого.

Сделаем это во view-функции `index()` приложения **homepage**:

```py
# homepage/views.py

from django.shortcuts import render

from ice_cream.models import IceCream

def index(request):
    template_name = 'homepage/index.html'
    # Запрашиваем нужные поля из базы данных:
    ice_cream_list = IceCream.objects.values(
        'id', 'title', 'price', 'description'
    ).filter(
        # Проверяем, что
        is_published=True,  # Сорт разрешён к публикации;
        is_on_main=True,  # Сорт разрешён к публикации на главной странице;
        category__is_published=True  # Категория разрешена к публикации.
    )

    context = {
        'ice_cream_list': ice_cream_list,
    }
    return render(request, template_name, context) 
```

А сортировку можно указать в описании самой модели:

```py
# ice_cream/models.py

...

class IceCream(PublishedModel):
    ...

    class Meta:
        ...
        # Сначала сортируем по полю output_order, 
        # а если у нескольких объектов значения output_order совпадают-- 
        # сортируем по title.
        ordering = ('output_order', 'title')
    
    def __str__(self):
        return self.title 
```

Теперь можно открыть главную страницу и проверить результат.

***
Осталось разобраться с отдельной страницей для сорта мороженого.

Начнём с изучения шаблона *detail.html*.

И в этот раз понадобятся почти все поля модели и проверка двух флагов, поэтому код view-функции может быть таким:

```py
...

def ice_cream_detail(request, pk):
    template_name = 'ice_cream/detail.html'
    ice_cream = get_object_or_404(
        IceCream.objects.filter(is_published=True, category__is_published=True),
        pk=pk
    )
    context = {
        'ice_cream': ice_cream,
    }
    return render(request, template_name, context)

...
```